
# Feast Client with RBAC  
### Example Using outside of Kubernetes for local testing

This notebook will  test Feast authentication outside of Kubernetes for local testing.

When running outside of Kubernetes, you need to manually set the service account token in the `LOCAL_K8S_TOKEN` environment variable. The token can be retrieved from a running pod using:  

```sh

kubectl exec <pod-name> -- cat /var/run/secrets/kubernetes.io/serviceaccount/token

```

To authenticate Feast externally, set the retrieved token as an environment variable:  

```sh

export LOCAL_K8S_TOKEN="your-service-account-token"

```  


## Test Cases
| User Type       | ServiceAccount               | RoleBinding Assigned | Expected Behavior in output                                |
|----------------|-----------------------------|----------------------|------------------------------------------------------------|
| **Read-Only**  | `feast-user-sa`              | `feast-reader`       | Can **read** from the feature store, but **cannot write**. |
| **Unauthorized** | `feast-unauthorized-user-sa` | _None_               | **Access should be denied** in `test.py`.                  |
| **Admin**      | `feast-admin-sa`             | `feast-writer`       | Can **read and write** feature store data.                 |

###  Feature Store settings

**The Operator create client feature store ConfigMap** containing the `feature_store.yaml `settings. We can retrieve it and port froward to local as we are testing locally.

In [2]:
!kubectl get configmap feast-sample-kubernetes-auth-client -n feast -o jsonpath='{.data.feature_store\.yaml}' > client/feature_repo/feature_store.yaml

Update the `client/feature_repo/feature_store.yaml` to look like below:

In [ ]:
project: feast_rbac
provider: local
offline_store:
    host: localhost
    type: remote
    port: 8081
online_store:
    path: http://localhost:8082
    type: remote
registry:
    path: localhost:8083
    registry_type: remote
auth:
    type: kubernetes
entity_key_serialization_version: 3

### The function below is executed to support the preparation of client testing.

Run Port Forwarding for All Services for local testing 

In [3]:
import subprocess

# Define services and their local ports
services = {
    "offline_store": ("feast-sample-kubernetes-auth-offline", 8081),
    "online_store": ("feast-sample-kubernetes-auth-online", 8082),
    "registry": ("feast-sample-kubernetes-auth-registry", 8083),
}

# Start port-forwarding for each service
port_forward_processes = {}
for name, (service, local_port) in services.items():
    cmd = f"kubectl port-forward svc/{service} -n feast {local_port}:80"
    process = subprocess.Popen(cmd, shell=True)
    port_forward_processes[name] = process
    print(f"Port forwarding {service} -> localhost:{local_port}")

Port forwarding feast-sample-kubernetes-auth-offline -> localhost:8081
Port forwarding feast-sample-kubernetes-auth-online -> localhost:8082
Port forwarding feast-sample-kubernetes-auth-registry -> localhost:8083


Forwarding from 127.0.0.1:8081 -> 8815
Forwarding from [::1]:8081 -> 8815
Forwarding from 127.0.0.1:8082 -> 6566
Forwarding from [::1]:8082 -> 6566
Forwarding from 127.0.0.1:8083 -> 6570
Forwarding from [::1]:8083 -> 6570


Function to retrieve a Kubernetes service account token and set it as an environment variable

In [4]:
import subprocess
import os

def get_k8s_token(service_account):
    namespace = "feast"

    if not service_account:
        raise ValueError("Service account name is required.")

    result = subprocess.run(
        ["kubectl", "create", "token", service_account, "-n", namespace],
        capture_output=True, text=True, check=True
    )

    token = result.stdout.strip()

    if not token:
        return None  # Silently return None if token retrieval fails

    os.environ["LOCAL_K8S_TOKEN"] = token
    return "Token Retrieved: ***** (hidden for security)"


**Generating training data**. The following test functions were copied from the `test_workflow.py` template but we added `try` blocks to print only 
the relevant error messages, since we expect to receive errors from the permission enforcement modules.

In [5]:
!cat client/feature_repo/test.py

import os

from feast import FeatureStore
from feast.data_source import PushMode
from datetime import datetime
import pandas as pd

# Initialize Feature Store
repo_path = os.getenv("FEAST_REPO_PATH", ".")
store = FeatureStore(repo_path=repo_path)

def fetch_historical_features_entity_df(store: FeatureStore, for_batch_scoring: bool):
    """Fetch historical features for training or batch scoring."""
    try:
        entity_df = pd.DataFrame.from_dict(
            {
                "driver_id": [1001, 1002, 1003],
                "event_timestamp": [
                    datetime(2021, 4, 12, 10, 59, 42),
                    datetime(2021, 4, 12, 8, 12, 10),
                    datetime(2021, 4, 12, 16, 40, 26),
                ],
                "label_driver_reported_satisfaction": [1, 5, 3],
                "val_to_add": [1, 2, 3],
                "val_to_add_2": [10, 20, 30],
            }
        )
        if for_batch_scoring:
            entity_df["event_timestamp"] = pd.to_datetim

In [6]:
import os

# Set the FEAST_REPO_PATH before importing check_permissions
os.environ["FEAST_REPO_PATH"] = "client/feature_repo"

### Test Read-Only Feast User 
**Step 1: Set the Token**

In [7]:
get_k8s_token("feast-user-sa")

'Token Retrieved: ***** (hidden for security)'

**Step 2: Test permission functions to validate permission on fetching online, offline or perform write operation**

In [8]:
from client.feature_repo.test import  check_permissions

# Call the function
#Run test.py script from pod to test RBAC for client-readonly-user.
# verify the logs for write operation will show below message 
# --- Write to Feature Store ---
#*** PERMISSION DENIED *** User lacks permission to modify the feature store.

check_permissions()



--- List feature views ---
Handling connection for 8083
Successfully listed 2 feature views:
  - driver_hourly_stats_fresh
  - driver_hourly_stats

--- Fetching Historical Features for Training ---
Handling connection for 8081
Successfully fetched training historical features:
    driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.287198  0.415895              540   
1           2            20   0.694777  0.622882              912   
2           3            30   0.107433  0.887705              491   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.287198            10.287198  
1             2.694777            20.6

**Step 3: Test permission functions to validate permission on fetching online feature views**

Required:
 - The Bearer token exported above as `LOCAL_K8S_TOKEN`

In [9]:
# Run Curl command to test the RBAC for client-readonly-user.
!curl -X POST http://localhost:8082/get-online-features -H "Content-Type: application/json" -H "Authorization: Bearer $LOCAL_K8S_TOKEN" -d '{"features": ["driver_hourly_stats:conv_rate","driver_hourly_stats:acc_rate"], "entities":{"driver_id": [1001, 1002]}}'

Handling connection for 8082
{"metadata":{"feature_names":["driver_id","acc_rate","conv_rate"]},"results":[{"values":[1001,1002],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[0.1861499696969986,0.7390778660774231],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]},{"values":[0.6494747400283813,0.6300525069236755],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]}]}

### Test Unauthorized Feast User 

**Step 1: Test permission functions to validate unauthorized user cant access any objects**

In [10]:
# Retrieve and store the token
get_k8s_token("feast-unauthorized-user-sa")

'Token Retrieved: ***** (hidden for security)'

In [11]:
check_permissions()


--- List feature views ---
No feature views found. You might not have access or they haven't been created.

--- Fetching Historical Features for Training ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Fetching Historical Features for Batch Scoring ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Write to Feature Store ---

*** PERMISSION DENIED *** User lacks permission to modify the feature store.

--- Fetching Online Features ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Feature Service ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Push Source ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Performing Push Source ---
Unexpected error while pushing event: Unable to find push source 'driver_stats_push_source'.


**Step 2: Run API request for Unauthorized User, Unauthorized user should not be able to even view the objects.**

Required:
 - The Bearer token exported above as `LOCAL_K8S_TOKEN`

In [12]:
# Run Curl command to test the RBAC for client-readonly-user.
!curl -X POST http://localhost:8082/get-online-features -H "Content-Type: application/json" -H "Authorization: Bearer $LOCAL_K8S_TOKEN" -d '{"features": ["driver_hourly_stats:conv_rate","driver_hourly_stats:acc_rate"], "entities":{"driver_id": [1001, 1002]}}'

Handling connection for 8082
"{\"module\": \"feast.errors\", \"class\": \"FeastPermissionError\", \"message\": \"Permission error:\\nPermission feast_user_permission denied execution of ['READ_ONLINE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-reader'],Permission feast_admin_permission denied execution of ['READ_ONLINE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-writer']\"}"

## Test Admin Feast User

**Step 1: Test permissions function to validate Admin user can access all objects**

In [13]:
# Retrieve and store the token
get_k8s_token("feast-admin-sa")

'Token Retrieved: ***** (hidden for security)'

In [14]:
check_permissions()


--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats_fresh
  - driver_hourly_stats

--- Fetching Historical Features for Training ---
Handling connection for 8081
Successfully fetched training historical features:
    driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.287198  0.415895              540   
1           2            20   0.694777  0.622882              912   
2           3            30   0.107433  0.887705              491   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.287198            10.287198  
1             2.694777            20.694777  
2             3.10743

**Step 2: Run API request for Admin User, Admin user should be able to perform all operations on the objects.**

Required:
 - The Bearer token exported above as `LOCAL_K8S_TOKEN`

In [16]:
# Run Curl command to test the RBAC for client-readonly-user.
!curl -X POST http://localhost:8082/get-online-features -H "Content-Type: application/json" -H "Authorization: Bearer $LOCAL_K8S_TOKEN" -d '{"features": ["driver_hourly_stats:conv_rate","driver_hourly_stats:acc_rate"], "entities":{"driver_id": [1001, 1002]}}'

Handling connection for 8082
{"metadata":{"feature_names":["driver_id","acc_rate","conv_rate"]},"results":[{"values":[1001,1002],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[0.1861499696969986,0.7390778660774231],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]},{"values":[0.6494747400283813,0.6300525069236755],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2025-05-20T11:00:00Z","2025-05-20T11:00:00Z"]}]}

 **Note:**
**Currently, remote materialization not available in Feast when using the Remote Client**
**Workaround: Consider using running it from pod like**
  
 `kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- bash -c 'feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")`


Terminate the process

In [17]:
for name, process in port_forward_processes.items():
    process.terminate()
    print(f"Stopped port forwarding for {name}")

Stopped port forwarding for offline_store
Stopped port forwarding for online_store
Stopped port forwarding for registry


[Next: Uninstall the Operator and all Feast objects](./04-uninstall.ipynb)